### Fake News Detector

#### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from tqdm import tqdm

#### Load Data Set

In [2]:
# put the path for the data set here
path = 'data_set/train_set.csv'
df = pd.read_csv(path, encoding='utf-8')
df = df.head(1000)  # For testing, use only the first 1000 rows
df = df.dropna(subset=['text'])  # Drop rows where 'text' is NaN

In [3]:
df.shape

(1000, 3)

In [4]:
df.head()

,label,source,text
0,1,translated,شى بوتين يوافق علا التعامل بشكل مناسب معا تجرب...
1,0,youtube,لكذب فوجه بنادم بركاو من لكذب
2,0,youtube,الحمدالله رب العالمين بسم الله مشاءالله و لا ح...
3,0,translated,حلفاء جباجبو وراء الهجمات على وزير داخلية ساحل...
4,0,translated,السيطرة على المعلومات تغير طريقها فالفترة الي ...


In [5]:
df.describe()

,label
count,1000.000000
mean,0.456000
std,0.498309
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1000 non-null   int64 
 1   source  1000 non-null   object
 2   text    1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [7]:
df.columns

Index(['label', 'source', 'text'], dtype='object')

In [8]:
# Count null rows
null_rows_count = df.isnull().sum().sum()
print(f"Number of null rows: {null_rows_count}")

# Count duplicated rows
duplicated_rows_count = df.duplicated().sum()
print(f"Number of duplicated rows: {duplicated_rows_count}")

Number of null rows: 0
Number of duplicated rows: 0


## NLP Processing

#### Text Cleaning

In [9]:
# Load custom Arabic stopwords from file
with open("./data_set/algerian_arabic_stopwords.txt", "r", encoding="utf-8") as f:
    custom_stopwords = set(word.strip() for word in f.readlines())

# -------------------------
# Text Normalization Function
def normalize_arabic(text):
    text = re.sub(r"[إأآٱ]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("[ًٌٍَُِّْ]", "", text)  # Remove diacritics
    return text

tqdm.pandas(desc="Normalizing Text")
df["normalized_text"] = df["text"].progress_apply(normalize_arabic)

# -------------------------
# Stopword Removal
def remove_stopwords(text):
    words = text.split()
    return " ".join([word for word in words if word not in custom_stopwords])

tqdm.pandas(desc="Removing Stopwords")
df["cleaned_text"] = df["normalized_text"].progress_apply(remove_stopwords)

# -------------------------
# Spelling Correction

# Save Cleaned Dataset
df.to_csv("cleaned_dataset.csv", index=False, encoding="utf-8-sig")
print("✅ Cleaning complete! Saved as 'cleaned_dataset.csv'")

Removing Stopwords: 100%|██████████| 1000/1000 [00:00<00:00, 125868.14it/s]

✅ Cleaning complete! Saved as 'cleaned_dataset.csv'


### Initialize Farasa Tools

In [10]:
# Initialize Farasa tools
segmenter = FarasaSegmenter()
pos_tagger = FarasaPOSTagger()
ner = FarasaNamedEntityRecognizer()
stemmer = FarasaStemmer()

In [11]:
# Load cleaned dataset
df = pd.read_csv("cleaned_dataset.csv")
df.head()

,label,source,text,normalized_text,cleaned_text
0,1,translated,شى بوتين يوافق علا التعامل بشكل مناسب معا تجرب...,شي بوتين يوافق علا التعامل بشكل مناسب معا تجرب...,شي بوتين يوافق علا التعامل بشكل مناسب معا تجرب...
1,0,youtube,لكذب فوجه بنادم بركاو من لكذب,لكذب فوجه بنادم بركاو من لكذب,لكذب فوجه بنادم بركاو لكذب
2,0,youtube,الحمدالله رب العالمين بسم الله مشاءالله و لا ح...,الحمدالله رب العالمين بسم الله مشاءالله و لا ح...,الحمدالله رب العالمين بسم الله مشاءالله حولا ق...
3,0,translated,حلفاء جباجبو وراء الهجمات على وزير داخلية ساحل...,حلفاء جباجبو وراء الهجمات علي وزير داخليه ساحل...,حلفاء جباجبو وراء الهجمات علي وزير داخليه ساحل...
4,0,translated,السيطرة على المعلومات تغير طريقها فالفترة الي ...,السيطره علي المعلومات تغير طريقها فالفتره الي ...,السيطره علي المعلومات تغير طريقها فالفتره تسبق...


In [ ]:
# -------------------------
# Tokenization
def tokenize_text(text):
    return segmenter.segment(text).split()

tqdm.pandas(desc="Tokenizing Text")
df["tokens"] = df["cleaned_text"].progress_apply(tokenize_text)
# -------------------------
# Save Processed Dataset
df.to_csv("processed_dataset.csv", index=False, encoding="utf-8-sig")
print("✅ Tokenization & Analysis complete! Saved as 'processed_dataset.csv'")

Tokenizing Text: 100%|██████████| 1000/1000 [54:45<00:00,  3.29s/it]

✅ Tokenization & Analysis complete! Saved as 'processed_dataset.csv'


In [13]:
preprocessing_df = pd.read_csv("processed_dataset.csv")
preprocessing_df.head()

,label,source,text,normalized_text,cleaned_text,tokens
0,1,translated,شى بوتين يوافق علا التعامل بشكل مناسب معا تجرب...,شي بوتين يوافق علا التعامل بشكل مناسب معا تجرب...,شي بوتين يوافق علا التعامل بشكل مناسب معا تجرب...,"['شي', 'بوتين', 'يوافق', 'علا', 'ال+تعامل', 'ب..."
1,0,youtube,لكذب فوجه بنادم بركاو من لكذب,لكذب فوجه بنادم بركاو من لكذب,لكذب فوجه بنادم بركاو لكذب,"['ل+كذب', 'فوج+ه', 'ب+نادم', 'ب+ركاو', 'ل+كذب']"
2,0,youtube,الحمدالله رب العالمين بسم الله مشاءالله و لا ح...,الحمدالله رب العالمين بسم الله مشاءالله و لا ح...,الحمدالله رب العالمين بسم الله مشاءالله حولا ق...,"['ال+حمدالله', 'رب', 'ال+عالم+ين', 'بسم', 'الل..."
3,0,translated,حلفاء جباجبو وراء الهجمات على وزير داخلية ساحل...,حلفاء جباجبو وراء الهجمات علي وزير داخليه ساحل...,حلفاء جباجبو وراء الهجمات علي وزير داخليه ساحل...,"['حلفاء', 'جباجبو', 'وراء', 'ال+هجم+ات', 'علي'..."
4,0,translated,السيطرة على المعلومات تغير طريقها فالفترة الي ...,السيطره علي المعلومات تغير طريقها فالفتره الي ...,السيطره علي المعلومات تغير طريقها فالفتره تسبق...,"['ال+سيطره', 'علي', 'ال+معلوم+ات', 'تغير', 'طر..."


In [19]:
# Apply stemming using the normalized_text
tqdm.pandas(desc="Stemming")
preprocessing_df["stemmed_tokens"] = preprocessing_df["normalized_text"].progress_apply(lambda text: stemmer.stem(text))
preprocessing_df["stemmed_tokens"] = preprocessing_df["stemmed_tokens"].apply(lambda text: text.split())

Stemming: 100%|██████████| 1000/1000 [1:06:08<00:00,  3.97s/it]


In [24]:
# Save results
preprocessing_df.to_csv("processed_with_stemming_pos.csv", index=False, encoding="utf-8-sig")
print("✅ Done! Output saved as 'processed_with_stemming_pos.csv'")

✅ Done! Output saved as 'processed_with_stemming_pos.csv'
